In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys
import warnings

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import dill

sys.path.append('..')
from seismiqb.batchflow import Dataset, Pipeline, B, V, C, L, F, D, DatasetIndex
from seismiqb.batchflow.models.tf import UNet, TFModel
from seismiqb import SeismicCropBatch, Geometry, parse_labels, make_histosampler
from seismiqb import make_geometries, make_labels, make_samplers

from tqdm import tqdm_notebook
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
path_data = '../../data/Facies/VU_ONGMK/Repaired.sgy'

path_label = '../../data/Facies/VU_ONGMK/il_xl_h.dill'
path_labels_saved = '../../data/Facies/VU_ONGMK/il_xl_h_model.dill'
path_geom_saved = '../../data/Facies/VU_ONGMK/geometry_model.dill'
path_samplers_saved = '../../data/Facies/VU_ONGMK/samplers_model.dill'

dsi = DatasetIndex([path_data])
ds = Dataset(dsi, batch_class = SeismicCropBatch)

In [ ]:
%%time
# geometries = make_geometries(dataset=ds, save_to=path_geom_saved)
geometries = make_geometries(dataset=ds, load_from=path_geom_saved)

# labels = make_labels(dataset=ds, load_from=path_label, save_to=path_labels_saved)
labels = make_labels(dataset=ds, load_from=path_labels_saved)

# samplers = make_samplers(dataset=ds, mode='hist', save_to=path_samplers_saved)
samplers = make_samplers(dataset=ds, mode='hist', load_from=path_samplers_saved)

In [ ]:
# Simple segmentation
model_config_simple = {'inputs': dict(cubes={'shape': (256, 256, 3)},
                                      masks={'name': 'targets', 'shape': (256, 256, 3)}),
                       'initial_block/inputs': 'cubes',
                       'body': dict(layout='cna pd cna pd cna tad tnad tnad t',
                                    filters=[4, 8, 16, 8, 4, 2, 3],
                                    kernel_size=[7]*7,
                                    strides=[2, 2, 1, 2, 2, 2, 2],
                                    activation=[tf.nn.elu]*7,
                                    pool_size=2, pool_strides=2,
                                    dropout_rate=.1),
                       'loss': 'mse',
                       'optimizer': 'Adam'
                        }

pipeline_config = {'model': TFModel,
                   'model_config': model_config_simple}

In [ ]:
def make_data(batch, **kwargs):
    data_x = []
    for cube in batch.data_crops:
        cube_ = np.swapaxes(cube, 0, 1)
        cube_ = np.swapaxes(cube_, 1, 2)
        data_x.append(cube_)
        
    data_y = []
    for cube in batch.mask_crops:
        cube_ = np.swapaxes(cube, 0, 1)
        cube_ = np.swapaxes(cube_, 1, 2)
        data_y.append(cube_.astype(int))
    return {"feed_dict": {'cubes': data_x,
                          'masks': data_y}}


train_pipeline = (Pipeline(config=pipeline_config)
                  .load_component(src=[D('geometries'), D('labels')],
                                  dst=['geometries', 'labels'])
                  .crop(points=L(ds.sampler.sample, 64), shape=[3, 256, 256])
                  .load_cubes(dst='data_crops')
                  .load_masks(dst='mask_crops')
                  .init_variable('loss_history', init_on_each_run=list)
                  .init_variable('current_loss')
                  .init_model('dynamic', C('model'), 'AE', C('model_config'))
                  .train_model('AE', 
                               fetches='loss', 
                               make_data=make_data,
                               save_to=V('current_loss'),
                               use_lock=True)
                  .update_variable('loss_history', 
                                   V('current_loss'), 
                                   mode='a'))

train_pipeline = train_pipeline << ds
loss_history = list()

In [ ]:
batch_size = 1
epochs = 2

train_pipeline.run(batch_size, n_epochs=epochs, drop_last=False, shuffle=False, bar=True)
loss_history.extend(train_pipeline.get_variable("loss_history"))

In [ ]:
plt.plot(loss_history)
plt.xlabel("Iterations"), plt.ylabel("Loss")
plt.show()

In [ ]:
test_pipeline = (Pipeline()
                 .load_component(src=[D('geometries'), D('labels')],
                                 dst=['geometries', 'labels'])
                 .crop(points=L(ds.sampler.sample, 16), shape=[3, 256, 256])
                 .load_cubes(dst='data_crops')
                 .load_masks(dst='mask_crops')
                 .import_model('AE', train_pipeline)
                 .init_variable('result', init_on_each_run=list()) 
                 .predict_model('AE', 
                                fetches=['predictions', 'cubes', 'masks'],
                                make_data=make_data, 
                                save_to=V('result'), mode='a')
                 )

test_pipeline = (test_pipeline) << ds

In [ ]:
test_batch = test_pipeline.next_batch(batch_size=1, n_epochs=1)

In [ ]:
for cube in range(4):
    iline = 1
    predicted_mask = test_pipeline.get_variable('result')[0][0][cube, :, :, iline].T
    img = test_pipeline.get_variable('result')[0][1][cube, :, :, iline].T
    masks = test_pipeline.get_variable('result')[0][2][cube, :, :, iline].T

    cv = 0.1
    fig, ax = plt.subplots(1, 2, figsize=(12, 7))

    ax[0].imshow(masks, cmap="Greens")
    ax[0].imshow(img, vmin=-cv, vmax=cv, cmap="gray", alpha=0.2)
    ax[0].set_title('Initial mask')

    ax[1].imshow(predicted_mask, cmap="Greens")
    ax[1].imshow(img, vmin=-cv, vmax=cv, cmap="gray", alpha=0.2)


    ax[1].set_title('Predicted mask')

    plt.show()

In [ ]:
test_batch.slice_points[0]

In [ ]:
def predict_on_iline(iline, pipeline, name, cube_name, shape):
    geom = pipeline.dataset.geometries[cube_name]
    x_ticks = geom.xlines_len // shape[1]
    h_ticks = geom.depth // shape[2]        
    
    points = []
    for x in range(x_ticks):
        for h in range(h_ticks):
            point = [cube_name,
                     (iline + shape[0]//2)/geom.ilines_len,
                     x * shape[1]/geom.xlines_len,
                     h * shape[2]/geom.depth]
            points.append(point)
    points = np.array(points, dtype=object)

    pred_pipeline = (Pipeline()
                     .load_component(src=[D('geometries'), D('labels')],
                                     dst=['geometries', 'labels'])
                     .crop(points=points, shape=shape)
                     .load_cubes(dst='data_crops')
                     .load_masks(dst='mask_crops')
                     .import_model(name, pipeline)
                     .init_variable('result', init_on_each_run=list()) 
                     .predict_model(name, 
                                    fetches=['predictions', 'cubes', 'masks'],
                                    make_data=make_data, 
                                    save_to=V('result'), mode='a')
                     )
    
    pred_pipeline = (pred_pipeline) << ds
    pred_batch = pred_pipeline.next_batch(batch_size=1, n_epochs=1)
    
    big_img = np.zeros((x_ticks*shape[1], h_ticks*shape[2]))
    big_mask = np.zeros((x_ticks*shape[1], h_ticks*shape[2]))
    big_pred = np.zeros((x_ticks*shape[1], h_ticks*shape[2]))
    
    for i, point in enumerate(points):
        point_ = np.array([point[2]*(geom.xlines_len - shape[1]),
                           point[3]*(geom.depth - shape[2])]).astype(int)
        
        img = pred_pipeline.get_variable('result')[0][1][i, :, :, 0]
        mask = pred_pipeline.get_variable('result')[0][2][i, :, :, 0]
        predicted_mask = pred_pipeline.get_variable('result')[0][0][i, :, :, 0]

        big_img[point_[0]: point_[0]+shape[1], point_[1]: point_[1]+shape[2]] = img
        big_mask[point_[0]: point_[0]+shape[1], point_[1]: point_[1]+shape[2]] = mask
        big_pred[point_[0]: point_[0]+shape[1], point_[1]: point_[1]+shape[2]] = predicted_mask

    
    cv = 0.1
    fig, ax = plt.subplots(2, 1, figsize=(12, 18))

    ax[0].imshow(big_mask.T, cmap="Greens")
    ax[0].imshow(big_img.T, vmin=-cv, vmax=cv, cmap="gray", alpha=0.2)
    ax[0].set_title('Initial mask')
    
    ax[1].imshow(big_pred.T, cmap="Greens")
    ax[1].imshow(big_img.T, vmin=-cv, vmax=cv, cmap="gray", alpha=0.2)
    ax[1].set_title('Predicted mask')
    plt.show()
    
    return pred_batch
    

In [ ]:
pred_batch = predict_on_iline(550, train_pipeline, 'AE', path_data, [3, 256, 256])

In [ ]:
class PixelShuffle(UNet):
    @classmethod
    def decoder_block(cls, inputs, filters, upsample=None, decoder=None, name='decoder', **kwargs):
        upsample = cls.fill_params('body/upsample', **upsample)
        decoder = cls.fill_params('body/decoder', **decoder)

        with tf.variable_scope(name):
            x, skip = inputs
            inputs = None
            x = conv_block(x, layout='Cna', kernel_size=3,
                            filters=filters*4, strides=1, activation=tf.nn.elu,
                            name='prepix')
            x = tf.nn.depth_to_space(x, block_size=2, name='pixel_shuffle')
#             x = cls.upsample(x, filters=filters, name='upsample', **{**kwargs, **upsample})
            x = cls.crop(x, skip, data_format=kwargs.get('data_format'))
            axis = cls.channels_axis(kwargs.get('data_format'))
            x = tf.concat((skip, x), axis=axis)
            x = conv_block(x, filters=filters, name='conv', **{**kwargs, **decoder})
        return x

In [ ]:
# Pixel-shuffle
model_config_ps = {
                    'inputs': dict(cubes={'shape': (256, 256, 3)},
                                   masks={'name': 'targets', 'shape': (256, 256, 3)}), 
                    'initial_block/inputs': 'cubes',
                    'body/filters': [8, 16, 32, 64],
                    'body/encoder': dict(layout='cna', kernel_size=3, activation=tf.nn.elu),
                    'body/downsample': dict(layout='pd', pool_size=2, pool_strides=2, dropout_rate=0.05),
                    'body/decoder': dict(layout='cna', kernel_size=3, activation=tf.nn.elu),
                    'body/upsample': dict(layout='tad', kernel_size=3, strides=2,
                                          dropout_rate=0.05, activation=tf.nn.elu),
                    'loss': 'mse',
                    'optimizer': 'Adam'
                     }

pipeline_config = {'model': PixelShuffle,
                   'model_config': model_config_ps}

In [ ]:
def make_data(batch, **kwargs):
    data_x = []
    for cube in batch.data_crops:
        cube_ = np.swapaxes(cube, 0, 1)
        cube_ = np.swapaxes(cube_, 1, 2)
        data_x.append(cube_)
        
    data_y = []
    for cube in batch.mask_crops:
        cube_ = np.swapaxes(cube, 0, 1)
        cube_ = np.swapaxes(cube_, 1, 2)
        data_y.append(cube_*10)
    return {"feed_dict": {'cubes': data_x,
                          'masks': data_y}}


train_pipeline = (Pipeline(config=pipeline_config)
                  .load_component(src=[D('geometries'), D('labels')],
                                  dst=['geometries', 'labels'])
                  .crop(points=L(ds.sampler.sample, 64), shape=[3, 256, 256])
                  .load_cubes(dst='data_crops')
                  .load_masks(dst='mask_crops')
                  .init_variable('loss_history', init_on_each_run=list)
                  .init_variable('current_loss')
                  .init_model('dynamic', C('model'), 'AE', C('model_config'))
                  .train_model('AE', 
                               fetches='loss', 
                               make_data=make_data,
                               save_to=V('current_loss'),
                               use_lock=True)
                  .update_variable('loss_history', 
                                   V('current_loss'), 
                                   mode='a'))

train_pipeline = train_pipeline << ds
loss_history = list()

In [ ]:
batch_size = 1
epochs = 5

train_pipeline.run(batch_size, n_epochs=epochs, drop_last=False, shuffle=False, bar=True)
loss_history.extend(train_pipeline.get_variable("loss_history"))